## Этап 1

In [1]:
import cudf.pandas  # Импортирование cuDF и активация его использования
cudf.pandas.install()  # Установка cuDF как основного интерфейса для pandas
import os
import pandas as pd

In [2]:
import os
import gdown

def download_file_if_not_exists(file_url, output_path):
    """Скачивает файл с Google Drive, если он ещё не существует в указанной директории."""
    # Проверка наличия файла
    if os.path.exists(output_path):
        print(f"Файл '{output_path}' уже существует.")
    else:
        print(f"Файл '{output_path}' не найден. Начинаю загрузку...")
        gdown.download(file_url, output_path, quiet=False)
        print(f"Файл '{output_path}' успешно загружен.")

# Указываем URL и путь к файлу
# file_url = 'https://drive.google.com/uc?id=15pofNbomaoUap41Rcn1uNGeiJIqFd2qe'
file_url = 'https://drive.google.com/uc?id=1alondqI-2IHo__mYU7KQz4Ip8ytYGHXg'
output_file_name = 'wildberries_reviews.csv'  # Укажите реальное имя файла, которое хотите сохранить
output_path = os.path.join(os.getcwd(), output_file_name)  # Полный путь к файлу

download_file_if_not_exists(file_url, output_path)


Файл '/workspace/wildberries_reviews.csv' уже существует.


In [3]:
# Путь к папке с CSV файлами
folder_path = './reviews_keywords/corrected_reviews'

# Получаем список всех файлов в папке
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Читаем и объединяем все CSV файлы в один датафрейм
df_list = [pd.read_csv(os.path.join(folder_path, file), index_col="id") for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=False)

combined_df.index = combined_df.index - 1
combined_df = pd.concat([pd.read_csv("wildberries_reviews.csv")[["corrected_text"]], combined_df], ignore_index=False)
# Выводим первые несколько строк объединенного датафрейма для проверки
combined_df.describe()


,corrected_text
count,517652
unique,331634
top,Мало мерит
freq,128


In [4]:
combined_df

,corrected_text
0,Работает хорошо.
1,"Пришло быстро, все целое на вид. Завтра буду и..."
2,"Купил на квадр для поднятия отвала, установка ..."
3,Лебёдка хорошая. Но в инструкции ни слова про ...
4,"Всё в комплекте, есть инструкция на русском яз..."
...,...
2169793,"Очень свежая и вкусная. Не очень сладкая, мягк..."
2169798,"Зимой была качественная хурма, сейчас муж попр..."
2169810,"Хурма вкусная, свежая, очень понравилась. Реко..."
2169829,"Хурма просто супер! Свежая, никакого сахара, а..."


In [5]:
df_raw_big = pd.read_csv("wildberries_reviews.csv.gz", compression="gzip").drop("Unnamed: 0", axis=1)
df_raw_big.head()

,review_full_text,review_rating,product,category,url
0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
4,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...


In [6]:
result = combined_df.merge(df_raw_big, left_index=True, right_index=True, how='right')
result.describe()

,review_rating
count,2.937733e+06
mean,4.592586e+00
std,1.036269e+00
min,1.000000e+00
25%,5.000000e+00
50%,5.000000e+00
75%,5.000000e+00
max,5.000000e+00


In [7]:
result['corrected_text'] = result['corrected_text'].fillna(result['review_full_text'])

In [8]:
result.head()

,review_full_text,review_rating,product,category,url,corrected_text
0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и..."
2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ..."
3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...
4,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Всё в комплекте, есть инструкция на русском яз..."


In [9]:
# Оставляем только по 5 записей для каждого уникального значения в столбце 'product'
result_limited = result.groupby('product').head(10).reset_index(drop=True)[:1000]


In [10]:
import spacy
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter

# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка модели и токенайзера от Сбербанка
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/sbert_large_nlu_ru')
model = AutoModel.from_pretrained('sberbank-ai/sbert_large_nlu_ru').to(device)

# Загрузка и настройка модели SpaCy
nlp = spacy.load("ru_core_news_lg")

# Пример загрузки данных в pandas DataFrame
df_raw = pd.read_csv("wildberries_reviews.csv", nrows=30000)
# df = df_raw[-500:-1]  # Отбор 500 записей для обработки
df = result_limited

# Преобразование pandas DataFrame в Hugging Face Dataset
dataset = Dataset.from_pandas(df)

import re

def clean_text(text):
    # Сначала заменяем все \n, \r, \t на пробел
    text = re.sub(r'[\n\r\t]+', ' ', text)
    
    # Удаляем лишние пробелы
    text = re.sub(r'\s{2,}', ' ', text)

    # Заменяем пробел и точку (если точка отсутствует)
    text = re.sub(r'(?<!\.)\s*\.\s*', '. ', text)  # Убедимся, что после замены есть точка и пробел
    text = re.sub(r'\s*\.\s*(?!\.)', ' ', text)  # Удаляем лишние пробелы перед точкой, если точка есть
    
    # Если текст заканчивается точкой, убираем её
    text = re.sub(r'\s*\.$', '', text)

    return text.strip()


# Функция для разбиения текста на предложения
def split_into_sentences(text):
    doc = nlp(clean_text(text))
    return [sent.text for sent in doc.sents]

# Применение функции для разбиения отзывов на предложения
def split_reviews_into_sentences(batch):
    batch['sentences'] = [split_into_sentences(text) for text in batch['corrected_text']]
    return batch

dataset = dataset.map(split_reviews_into_sentences, batched=True, batch_size=8)

# Преобразуем Dataset обратно в pandas DataFrame
df = dataset.to_pandas()

# Выполним explode по столбцу с предложениями
df_exploded = df.explode('sentences').reset_index(drop=True)

# Удаляем лишние столбцы, которые появились после explode
df_exploded = df_exploded.drop(columns=[col for col in df_exploded.columns if col.startswith('__index_level_')])

# Преобразуем DataFrame обратно в Hugging Face Dataset
dataset_exploded = Dataset.from_pandas(df_exploded)

# Функция для вычисления эмбеддингов для каждого предложения
def compute_sentence_embeddings(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

# Функция для вычисления эмбеддингов для каждого предложения после explode
def compute_embeddings_after_explode(batch):
    sentences = batch['sentences']
    embeddings = compute_sentence_embeddings(sentences)
    batch['sentence_embeddings'] = embeddings
    return batch

# Применение функции
dataset = dataset_exploded.map(compute_embeddings_after_explode, batched=True, batch_size=8)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1389 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [11]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoTokenizer, AutoModel
import nltk
from nltk.corpus import stopwords
import spacy
from tqdm import tqdm
import logging

# Отключение параллелизма в токенайзере Hugging Face
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка модели и токенайзера от Сбербанка
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/sbert_large_nlu_ru')
model = AutoModel.from_pretrained('sberbank-ai/sbert_large_nlu_ru').to(device)

# Настройка логирования
logging.basicConfig(filename='./reviews_keywords/clustering.log', 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Загрузка модели spaCy для русского языка
nlp = spacy.load("ru_core_news_lg")

# Установка стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# Функция для лемматизации текста с использованием spaCy
def lemmatize_sentences(sentences):
    lemmatized_sentences = []
    for doc in tqdm(nlp.pipe(sentences, batch_size=32, n_process=-1), total=len(sentences), desc="Лемматизация"):
        lemmatized_sentences.append(" ".join([token.lemma_ for token in doc]))
    return lemmatized_sentences

# Функция для вычисления эмбеддингов с дополнительной проверкой
def compute_sentence_embeddings(sentences):
    if not sentences:
        return np.array([])  # Возвращаем пустой массив, если список предложений пуст
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

# Функция для привязки предложений к маскам с логированием и метками
def assign_to_masks(sentences, mask_embeddings, mask_names, threshold=0.65):
    labeled_sentences = []

    for sentence in sentences:
        sentence_emb = compute_sentence_embeddings([sentence])
        similarities = [np.max(cosine_similarity(sentence_emb, mask_emb)) for mask_emb in mask_embeddings]
        max_similarity = np.max(similarities)
        best_mask_index = np.argmax(similarities)

        if max_similarity > threshold:
            mask_name = mask_names[best_mask_index]
            logging.info(f"Предложение '{sentence}' привязано к маске '{mask_name}' с похожестью {max_similarity:.2f}")
            labeled_sentences.append((sentence, 0))  # Привязано к маске, метка 0
        else:
            logging.info(f"Предложение '{sentence}' не привязано ни к одной маске")
            labeled_sentences.append((sentence, 1))  # Не привязано к маске, метка 1

    return labeled_sentences

# Обработка коротких предложений и специфичных фраз
def classify_short_sentences(sentences, mask_words, min_words=2, max_words=4):
    short_sentences = []
    
    for sentence in sentences:
        words = sentence.split()
        if min_words <= len(words) <= max_words:
            match_count = sum(1 for word in words if word in mask_words)
            if match_count > 0:
                short_sentences.append(sentence)
        # Дополнительное условие для фраз типа "Очки понравились"
        if "понравились" in sentence or "неплохие" in sentence or "отличные" in sentence:
            short_sentences.append(sentence)
    
    return short_sentences

# Основной процесс с проверками и прогресс-баром
def process_reviews(df_exploded, mask_embeddings, mask_names, mask_words, threshold=0.6, eps=0.25, min_samples=3):
    final_result = pd.DataFrame()

    # tqdm для прогресса по продуктам
    for product_name, group in tqdm(df_exploded.groupby('product'), desc="Обработка продуктов"):
        all_sentences = group['sentences'].tolist()

        # Пропуск, если нет предложений
        if not all_sentences:
            logging.info(f"Пропуск продукта {product_name}, так как нет предложений.")
            continue

        # Привязка предложений к маскам и присвоение меток
        labeled_sentences = assign_to_masks(all_sentences, mask_embeddings, mask_names, threshold)

        # Отдельный список для меток и предложений
        sentences, labels = zip(*labeled_sentences)

        # Обработка коротких предложений и специфичных фраз
        short_sentences = classify_short_sentences(sentences, mask_words)
        short_labeled_sentences = [(sentence, 2) for sentence in short_sentences]

        # Исключение коротких предложений из основного списка (чтобы они не дублировались)
        filtered_sentences_labels = [(s, l) for s, l in zip(sentences, labels) if s not in short_sentences]

        # Check if filtered_sentences_labels is empty before unpacking
        if filtered_sentences_labels:
            sentences, labels = zip(*filtered_sentences_labels)
        else:
            sentences, labels = [], []

        # Adding short labeled sentences to the DataFrame
        df_labels = pd.DataFrame({
            'product': product_name,
            'sentence': list(sentences) + [s for s, l in short_labeled_sentences],
            'label': list(labels) + [l for s, l in short_labeled_sentences]
        })

        final_result = pd.concat([final_result, df_labels], ignore_index=True)

    return final_result


# Лемматизация и создание эмбеддингов для масок
def prepare_mask_embeddings(mask_phrases):
    lemmatized_masks = lemmatize_sentences(mask_phrases)
    return compute_sentence_embeddings(lemmatized_masks)

# Создание списка всех слов из масок
def create_mask_words(mask_phrases):
    all_words = []
    for phrase in mask_phrases:
        all_words.extend(phrase.split())
    lemmatized_words = lemmatize_sentences(all_words)
    return set(lemmatized_words)


quality_phrases = [
    r'прекрасная вещь', r'замечательная вещь', 
    r'все пришло в идеальном состоянии', r'товар в отличном состоянии', 
    r'без повреждений', r'упаковка целая', r'товар без дефектов', 
    r'все дошло целым', r'доставка без повреждений', r'идеальное состояние',
    r'очень доволен', r'очень довольна', r'товар понравился', r'качество понравилось'
]


functionality_phrases = [
    r'работает отлично', r'работает хорошо', r'всё работает', r'функции выполняет', r'функциональный', 
    r'функции справляются', r'с задачей справился', r'справляется с задачей', r'задачу свою выполнил', 
    r'справился на отлично', r'со своими функциями справляется', r'задачу выполнил',
    r'доволен функциональностью', r'довольна функциональностью', r'функционал понравился'
]

gratitude_phrases = [
    r'спасибо', r'рекомендую', r'советую', r'продавец молодец', r'благодарен', r'благодарю', 
    r'советую к покупке', r'спасибо большое', r'всем советую', r'спасибо за товар', 
    r'спасибо продавцу', r'благодарю за товар', r'большое спасибо', r'очень благодарен', 
    r'спасибо за доставку', r'огромное спасибо', r'спасибо за качественный товар', 
    r'продавцу огромное спасибо', r'спасибо за оперативность', r'спасибо вам', 
    r'благодарен за товар', r'спасибо, всё хорошо', r'продавец молодец', r'спасибо за хорошее обслуживание',
    r'доволен сервисом', r'довольна сервисом'
]

delivery_phrases = [
    r'пришел быстро', r'быстрая доставка', r'пришел вовремя', r'заказ пришел целый и вовремя', 
    r'пришел целый', r'доставка вовремя', r'все пришло целым', r'товар пришел целым', r'пришел в срок',
    r'пришел вовремя и целым', r'получил заказ вовремя', r'доставка - во!', r'все пришло как надо', 
    r'пришел в полном порядке', r'отличная упаковка', r'все дошло целым', r'упаковано на совесть', 
    r'крутая упаковка', r'доволен доставкой', r'довольна доставкой'
]

confirmation_phrases = [
    r'всё соответствует', r'всё как в описании', r'всё как заявлено', r'соответствует описанию', 
    r'всё целое', r'всё в комплекте', r'всё норм', r'всё хорошо', r'как всегда', r'без проблем', 
    r'нормально упаковано', r'нормально', r'всё норм', r'полностью доволен', r'полностью довольна', 
    r'всё понравилось'
]

simple_statements_phrases = [
    r'хорошая вещь', r'классная вещь', r'отличная вещь', r'удобно', r'нормально', r'работает', 
    r'работает отлично', r'работает хорошо', r'всё нормально', r'всё работает', r'всё ок', 
    r'всё окей', r'супер', r'класс', r'норм', r'отлично', r'хорошо', r'идеально', r'👍', r'👏', 
    r'😆', r'🔥', r'💯', r'класс', r'все супер', r'😊', r'доволен', r'довольна', 
    r'понравилось', "😊", "👍", "😍", "😂", "🛍️", "💯", "😆", "😁", "👏", "🔥",
    "🥰", "😎", "🤩", "❤️", "🤔", "🙌", "😜", "😉", "🤗", "😅",
    "👀", "🤷", "😋", "💖", "🌟", "😇", "😘", "🎉", "💪", "💥",
    "👌", "😄", "👋", "😏", "🙏", "🤝", "✨", "🤓", "🌸", "😌",
    "🥳", "🎁", "😑", "😳", "🙈", "😤", "👑", "😢", "🤤", "🤞"
]

# Определение масок и их эмбеддингов
mask_names = [
    "quality_phrases",
    "functionality_phrases",
    "gratitude_phrases",
    "delivery_phrases",
    "confirmation_phrases",
    "simple_statements_phrases"
]

mask_embeddings = [
    compute_sentence_embeddings(gratitude_phrases),
    compute_sentence_embeddings(delivery_phrases),
    compute_sentence_embeddings(confirmation_phrases),
    compute_sentence_embeddings(simple_statements_phrases),
    compute_sentence_embeddings(quality_phrases),
    compute_sentence_embeddings(functionality_phrases),
]

# Создание списка всех лемматизированных слов из масок
mask_words = create_mask_words(
    gratitude_phrases + delivery_phrases + confirmation_phrases + simple_statements_phrases + quality_phrases + functionality_phrases
)

# Вызов основной функции с эмбеддингами масок и логированием
final_result = process_reviews(df_exploded, mask_embeddings, mask_names, mask_words)

# Показать результат
display(final_result[['product', 'sentence', 'label']])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Обработка продуктов: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 107/107 [00:29<00:00,  3.63it/s]


,product,sentence,label
0,3TON / Очиститель карбюратора и дроссельной за...,Крышка на очистителе расколота и держится толь...,1
1,3TON / Очиститель карбюратора и дроссельной за...,Ок,0
2,3TON / Очиститель карбюратора и дроссельной за...,"Пришло вовремя, отлично упакованным Муж сказал...",0
3,3TON / Очиститель карбюратора и дроссельной за...,Товар пока не проверяли За вскрытую упаковку с...,0
4,3TON / Очиститель карбюратора и дроссельной за...,"Упаковано в коробку все целое приехало, отличн...",0
...,...,...,...
1384,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,"Все отлично, подошли без проблем!",0
1385,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,Соответствует описанию!,0
1386,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,Пришли качество 🔥пока ещё не ставил,1
1387,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,"Все встало без проблем, уаз поднялся на 3 см К...",0


In [12]:
import gc
gc.collect()

12095

In [13]:
# import os
# import numpy as np
# import pandas as pd
# from sklearn.cluster import DBSCAN
# from sklearn.metrics.pairwise import cosine_similarity
# import torch
# from transformers import AutoTokenizer, AutoModel
# import nltk
# from nltk.corpus import stopwords
# import spacy
# from tqdm import tqdm
# import logging

# # Отключение параллелизма в токенайзере Hugging Face
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# # Устройство (GPU или CPU)
# device = torch.device("cpu")  # Ensure everything runs on CPU

# # Загрузка модели и токенайзера от Сбербанка
# tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/sbert_large_nlu_ru')
# model = AutoModel.from_pretrained('sberbank-ai/sbert_large_nlu_ru').to(device)

# # Настройка логирования
# logging.basicConfig(filename='./reviews_keywords/clustering.log', 
#                     level=logging.INFO, 
#                     format='%(asctime)s - %(levelname)s - %(message)s')

# # Загрузка модели spaCy для русского языка
# nlp = spacy.load("ru_core_news_lg")

# # Установка стоп-слов
# nltk.download('stopwords')
# stop_words = set(stopwords.words('russian'))

In [14]:
# # Функция для вычисления центра кластера (центроида)
# def find_centroid(embeddings):
#     return np.mean(embeddings, axis=0)

# # Функция для вычисления эмбеддингов
# def compute_sentence_embeddings(sentences):
#     inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         # Получаем скрытые состояния
#         hidden_states = outputs.hidden_states[-1]
#     embeddings = hidden_states.mean(dim=1).cpu().numpy()
#     return embeddings

# # Функция для нахождения ключевой мысли в кластере
# def extract_key_thought(cluster_sentences):
#     sentences = cluster_sentences.split(" | ")
    
#     embeddings = compute_sentence_embeddings(sentences)
    
#     centroid = find_centroid(embeddings)
#     similarities = cosine_similarity(embeddings, [centroid])
#     key_sentence_index = np.argmax(similarities)
    
#     return sentences[key_sentence_index]

# # Функция для подсчета количества слов в каждом кластере
# def count_words(cluster_sentences):
#     words = cluster_sentences.split()
#     return len(words)


# # Функция для повторной кластеризации крупных кластеров
# def recluster_large_cluster(cluster_sentences, eps=0.1, min_samples=2):
#     sentences = cluster_sentences.split(" | ")
    
#     embeddings = compute_sentence_embeddings(sentences)
    
#     re_clustering = DBSCAN(eps=eps, min_samples=min_samples, metric="cosine").fit(embeddings)
    
#     re_cluster_dict = {}
#     for idx, label in enumerate(re_clustering.labels_):
#         if label == -1:
#             continue
#         label_str = str(label)
#         if label_str not in re_cluster_dict:
#             re_cluster_dict[label_str] = []
#         re_cluster_dict[label_str].append(sentences[idx])
    
#     return [" | ".join(cluster) for cluster in re_cluster_dict.values()]

# # Рекурсивная функция для кластеризации крупных кластеров
# def recursive_clustering(cluster_sentences, threshold, eps=0.25, min_samples=3, min_eps=0.05):
#     current_eps = eps
#     new_clusters = [cluster_sentences]

#     while True:
#         next_clusters = []
#         reclustered_any = False
        
#         for cluster in new_clusters:
#             if count_words(cluster) > threshold:
#                 while current_eps >= min_eps:
#                     reclustered = recluster_large_cluster(cluster, eps=current_eps, min_samples=min_samples)
#                     if len(reclustered) > 1:
#                         next_clusters.extend(reclustered)
#                         reclustered_any = True
#                         break  # Кластер успешно разделен, выходим из внутреннего цикла
#                     else:
#                         current_eps -= 0.02  # Уменьшаем eps и пробуем снова
                
#                 if len(reclustered) == 1:
#                     # Если кластер так и не был разделен, добавляем его обратно
#                     next_clusters.append(cluster)
#             else:
#                 next_clusters.append(cluster)
        
#         new_clusters = next_clusters
        
#         if not reclustered_any:
#             break
    
#     return new_clusters

# # Основной процесс кластеризации по товарам
# df_clusters = pd.DataFrame()
# label_col = "label"
# sentence_col = "sentence"
# for label in final_result[label_col].unique():  # Added tqdm here
#     print(label)
#     label_df = final_result[final_result[label_col] == label]
#     all_sentences = label_df[sentence_col].tolist()
#     print(all_sentences)
#     # Обработка предложений без разделения на батчи
#     all_embeddings = compute_sentence_embeddings(all_sentences)
#     print(all_embeddings)

#     # Прогресс-бар для начальной кластеризации
#     clustering = DBSCAN(eps=0.25, min_samples=3, metric="cosine").fit(all_embeddings)
#     print(clustering)

#     cluster_dict = {}
#     for idx, label in tqdm(enumerate(clustering.labels_), desc=f"Organizing clusters for {label}"):
#         if label == -1:
#             continue
#         label_str = str(label)
#         if label_str not in cluster_dict:
#             cluster_dict[label_str] = set()
#         cluster_dict[label_str].add(all_sentences[idx])

#     clusters = [" | ".join(sentences) for sentences in cluster_dict.values()]
#     threshold = np.min([np.mean([count_words(cluster) for cluster in clusters]) * 1.5  ,  450])

#     final_clusters = []
#     for cluster in tqdm(clusters, desc="Recursive clustering"):
#         final_clusters.extend(recursive_clustering(cluster, threshold))

#     df_exploded_sorted = pd.DataFrame({'cluster_sentences': final_clusters})
#     df_exploded_sorted['word_count'] = df_exploded_sorted['cluster_sentences'].apply(count_words)
#     df_exploded_sorted['key_thought'] = df_exploded_sorted['cluster_sentences'].apply(extract_key_thought)

#     df_exploded_sorted = df_exploded_sorted.sort_values(by='word_count', ascending=False)

#     df_clusters = pd.concat([df_clusters, df_exploded_sorted], ignore_index=True)

# # Показать результат
# display(df_clusters[['cluster_sentences', 'key_thought', 'word_count']])


## Этап 2

In [14]:
import cudf.pandas  # Импортирование cuDF и активация его использования
cudf.pandas.install()  # Установка cuDF как основного интерфейса для pandas
import pandas as pd  # Импортирование pandas после установки cuDF

import os
import yaml
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from tqdm import tqdm
from IPython.display import display
import numpy as np

In [15]:
clustered_result = pd.read_csv("./reviews_keywords/cluster_result.csv").drop("Unnamed: 0", axis=1)

In [16]:
clustered_result

,label,cluster_id,cluster_sentences,key_thought,word_count
0,0,0,"['Оч понравилась!', 'Добротная крышка, ручка д...","Стильный дизайн,качественно выполнен Пришёл хо...",179535
1,0,1,"['Доставка быстрая, пока ещё ничего не клеил, ...","Доставка быстрая, пока ещё ничего не клеил, ду...",44
2,0,2,"['абсолютно', 'абсолютно', 'вполне', 'более чем']",абсолютно,5
3,0,3,"['Очень приятно😜', 'Очень красивые👍🏻', 'Очень ...",Очень приятно😜,21
4,0,4,"['То что искала, спксиьо продавцу!', 'То что и...","То что искала, спксиьо продавцу!",15
...,...,...,...,...,...
668,2,64,"['Пришлось выкупать и ремонтировать!', 'Пришло...",Пришлось выкупать и ремонтировать!,20
669,2,65,"['Хороший результат, без сварки', 'Хороший рез...","Хороший результат, без сварки",12
670,2,68,"['Яркий, очень аккуратно сделан👍🏻', 'Яркий, оч...","Яркий, очень аккуратно сделан👍🏻",16
671,2,69,"['Пришёл с механическими повреждениями', 'Приш...",Пришёл с механическими повреждениями,16


In [17]:
import re

def extract_list_from_string(s):
    # Регулярное выражение для поиска элементов списка внутри строки
    matches = re.findall(r'\[\'(.*?)\'\]', s)
    
    # Если нашли соответствие, разделяем элементы по запятой и возвращаем список
    if matches:
        return [item.strip() for item in matches[0].split("', '")]
    return s

# Применяем функцию ко всей колонке
clustered_result['cluster_sentences'] = clustered_result['cluster_sentences'].apply(lambda x: extract_list_from_string(str(x)))

# Проверяем результат
clustered_result


,label,cluster_id,cluster_sentences,key_thought,word_count
0,0,0,"[Оч понравилась!, Добротная крышка, ручка доба...","Стильный дизайн,качественно выполнен Пришёл хо...",179535
1,0,1,"[Доставка быстрая, пока ещё ничего не клеил, д...","Доставка быстрая, пока ещё ничего не клеил, ду...",44
2,0,2,"[абсолютно, абсолютно, вполне, более чем]",абсолютно,5
3,0,3,"[Очень приятно😜, Очень красивые👍🏻, Очень хорош...",Очень приятно😜,21
4,0,4,"[То что искала, спксиьо продавцу!, То что иска...","То что искала, спксиьо продавцу!",15
...,...,...,...,...,...
668,2,64,"[Пришлось выкупать и ремонтировать!, Пришлось ...",Пришлось выкупать и ремонтировать!,20
669,2,65,"[Хороший результат, без сварки, Хороший резуль...","Хороший результат, без сварки",12
670,2,68,"[Яркий, очень аккуратно сделан👍🏻, Яркий, очень...","Яркий, очень аккуратно сделан👍🏻",16
671,2,69,"[Пришёл с механическими повреждениями, Пришёл ...",Пришёл с механическими повреждениями,16


In [18]:
clustered_result.loc[clustered_result.label == 2, "label"] = 0

In [19]:
# Группировка по метке `label` и объединение массивов текстов
grouped_result = clustered_result.groupby('label').agg({
    'cluster_sentences': lambda x: sum(x, []),  # Объединяем списки текстов
    'cluster_id': 'first',  # Можно оставить любой cluster_id, так как они больше не будут уникальными
    'key_thought': 'first',  # Оставляем первую ключевую мысль
    'word_count': 'sum'  # Суммируем количество слов
}).reset_index()
# Подсчет итогового количества строк в массиве текстов
grouped_result['total_sentences'] = grouped_result['cluster_sentences'].apply(len)
grouped_result

,label,cluster_sentences,cluster_id,key_thought,word_count,total_sentences
0,0,"[Оч понравилась!, Добротная крышка, ручка доба...",0,"Стильный дизайн,качественно выполнен Пришёл хо...",208571,24236
1,1,"[Удобная ручка, плотный прозрачный материал - ...",0,Олимпийка отличного качества вначале купила сы...,796358,33113


In [20]:
grouped_result_exploded = grouped_result.explode('cluster_sentences')[['label', 'cluster_sentences']].drop_duplicates()
grouped_result_exploded

,label,cluster_sentences
0,0,Оч понравилась!
0,0,"Добротная крышка, ручка добавляет удобство - н..."
0,0,Хороший паяльник Качество за такие деньги лучш...
0,0,"Отличный паяльник, шнур не короткий, работает ..."
0,0,Рекомендую
...,...,...
1,1,"Хорошая открытка, но подставки хрупкие, одна с..."
1,1,Очень жаль что вторая форма пришла больше разм...
1,1,Очень жаль что вторая форма пришла больше разм...
1,1,Вот такие кирпичики у нас получились


In [21]:
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import logging
from transformers import TrainerCallback

# Разделение данных на тренировочную и валидационную выборки
train_df, val_df = train_test_split(grouped_result_exploded, test_size=0.1, random_state=42, stratify=grouped_result_exploded['label'])

# Пример данных для обучения
train_texts = train_df['cluster_sentences'].tolist()
train_labels = train_df['label'].tolist()

# Пример данных для валидации
val_texts = val_df['cluster_sentences'].tolist()
val_labels = val_df['label'].tolist()

# Загрузка токенизатора и модели
tokenizer = BertTokenizerFast.from_pretrained('sberbank-ai/sbert_large_nlu_ru')
model = BertForSequenceClassification.from_pretrained('sberbank-ai/sbert_large_nlu_ru', num_labels=2).to('cuda')

# Подготовка данных
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)  # Оставляем метки на CPU
        }

def compute_metrics(eval_pred, threshold=0.4):  # Устанавливаем порог ниже 0.5
    logits, labels = eval_pred
    predictions = (logits[:, 1] > threshold).astype(int)  # Применение порога
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Создание датасетов
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_len=128)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_len=128)

# Создание DataLoader-ов с использованием нескольких воркеров
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

# Определение аргументов для тренировки
training_args = TrainingArguments(
    output_dir='./reviews_keywords/results',
    num_train_epochs=2,  # Уменьшение количества эпох
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=200,  # Уменьшение количества шагов прогрева
    weight_decay=0.01,
    logging_dir='./reviews_keywords/logs',
    logging_steps=10,
    evaluation_strategy="steps",  # Валидация на каждом шаге
    eval_steps=120,  # Валидация каждые 10 шагов
    fp16=True,  # Использование 16-битной точности
    gradient_accumulation_steps=2,  # Увеличение шага аккумуляции градиентов
)

# Настройка логирования
logging.basicConfig(filename='./reviews_keywords/clustering.log', 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Создание коллбэка для дополнительного логирования
class LogCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            logging.info(f"Log at step {state.global_step}: {logs}")

# Trainer API от Hugging Face
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,  # Добавление метрик для оценки
    callbacks=[LogCallback()]  # Включение коллбэка для логирования
)

# Запуск обучения с валидацией
trainer.train()

# Сохранение дообученной модели
model.save_pretrained('./reviews_keywords/fine_tuned_model_10')
tokenizer.save_pretrained('./reviews_keywords/fine_tuned_model_10')


## Кластеризация и всякие тесты

In [22]:
# import numpy as np
# import pandas as pd
# from sklearn.cluster import DBSCAN
# from sklearn.metrics.pairwise import cosine_similarity
# import torch
# from transformers import AutoTokenizer, AutoModel
# import nltk
# from nltk.corpus import stopwords
# import spacy
# from tqdm import tqdm
# import logging

# # Устройство (GPU или CPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Загрузка модели и токенайзера от Сбербанка
# tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/sbert_large_nlu_ru')
# model = AutoModel.from_pretrained('sberbank-ai/sbert_large_nlu_ru').to(device)

# # Настройка логирования
# logging.basicConfig(filename='./reviews_keywords/clustering.log', 
#                     level=logging.INFO, 
#                     format='%(asctime)s - %(levelname)s - %(message)s')

# # Загрузка модели spaCy для русского языка
# nlp = spacy.load("ru_core_news_lg")

# # Установка стоп-слов
# nltk.download('stopwords')
# stop_words = set(stopwords.words('russian'))

# # Функция для вычисления эмбеддингов с дополнительной проверкой
# def compute_sentence_embeddings(sentences):
#     if not sentences:
#         return np.array([])  # Возвращаем пустой массив, если список предложений пуст
#     inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

# # Функция для привязки предложений к маскам с логированием
# def assign_to_masks(sentences, mask_embeddings, mask_names, threshold=0.65):
#     assigned_sentences = []
#     unassigned_sentences = []

#     for sentence in sentences:
#         sentence_emb = compute_sentence_embeddings([sentence])
#         similarities = [np.max(cosine_similarity(sentence_emb, mask_emb)) for mask_emb in mask_embeddings]
#         max_similarity = np.max(similarities)
#         best_mask_index = np.argmax(similarities)

#         if max_similarity > threshold:
#             assigned_sentences.append(sentence)
#             mask_name = mask_names[best_mask_index]
#             logging.info(f"Предложение '{sentence}' привязано к маске '{mask_name}' с похожестью {max_similarity:.2f}")
#         else:
#             unassigned_sentences.append(sentence)
#             logging.info(f"Предложение '{sentence}' не привязано ни к одной маске")

#     return assigned_sentences, unassigned_sentences

# # Функция для кластеризации оставшихся предложений
# def cluster_remaining_sentences(sentences, eps=0.25, min_samples=3):
#     embeddings = compute_sentence_embeddings(sentences)
#     clustering = DBSCAN(eps=eps, min_samples=min_samples, metric="cosine").fit(embeddings)
    
#     clusters = {}
#     for idx, label in enumerate(clustering.labels_):
#         if label == -1:
#             continue
#         if label not in clusters:
#             clusters[label] = []
#         clusters[label].append(sentences[idx])
    
#     return clusters

# # Основной процесс с проверками и прогресс-баром
# def process_reviews(df_exploded, mask_embeddings, mask_names, threshold=0.6, eps=0.25, min_samples=3):
#     final_result = pd.DataFrame()

#     # tqdm для прогресса по продуктам
#     for product_name, group in tqdm(df_exploded.groupby('product'), desc="Обработка продуктов"):
#         all_sentences = group['sentences'].tolist()

#         # Пропуск, если нет предложений
#         if not all_sentences:
#             logging.info(f"Пропуск продукта {product_name}, так как нет предложений.")
#             continue

#         # Привязка предложений к маскам
#         assigned_sentences, unassigned_sentences = assign_to_masks(all_sentences, mask_embeddings, mask_names, threshold)

#         # Пропуск, если нет предложений для кластеризации
#         if not unassigned_sentences:
#             logging.info(f"Пропуск продукта {product_name}, так как все предложения были привязаны к маскам.")
#             continue

#         # Кластеризация оставшихся предложений
#         remaining_clusters = cluster_remaining_sentences(unassigned_sentences, eps, min_samples)

#         # Подготовка результатов с tqdm для прогресса по кластерам
#         for cluster_id, sentences in tqdm(remaining_clusters.items(), desc=f"Кластеризация для {product_name}"):
#             if not sentences:  # Пропуск пустых кластеров
#                 continue
#             cluster_text = " | ".join(sentences)
#             key_thought = extract_key_thought(cluster_text)  # Используем ранее описанную функцию

#             df_cluster = pd.DataFrame({
#                 'product': product_name,
#                 'cluster_id': cluster_id,
#                 'cluster_sentences': [cluster_text],
#                 'key_thought': [key_thought],
#                 'word_count': [count_words(cluster_text)]
#             })

#             final_result = pd.concat([final_result, df_cluster], ignore_index=True)

#     return final_result

# # Функция для нахождения ключевой мысли в кластере
# def extract_key_thought(cluster_sentences):
#     sentences = cluster_sentences.split(" | ")
#     embeddings = compute_sentence_embeddings(sentences)
    
#     centroid = find_centroid(embeddings)
#     similarities = cosine_similarity(embeddings, [centroid])
#     key_sentence_index = np.argmax(similarities)
    
#     return sentences[key_sentence_index]

# # Функция для вычисления центра кластера (центроида)
# def find_centroid(embeddings):
#     return np.mean(embeddings, axis=0)

# # Функция для подсчета количества слов в каждом кластере
# def count_words(cluster_sentences):
#     words = cluster_sentences.split()
#     return len(words)

# quality_phrases = [
#     r'прекрасная вещь', r'замечательная вещь', 
#     r'все пришло в идеальном состоянии', r'товар в отличном состоянии', 
#     r'без повреждений', r'упаковка целая', r'товар без дефектов', 
#     r'все дошло целым', r'доставка без повреждений', r'идеальное состояние'
# ]

# functionality_phrases = [
#     r'работает отлично', r'работает хорошо', r'всё работает', r'функции выполняет', r'функциональный', 
#     r'функции справляются', r'с задачей справился', r'справляется с задачей', r'задачу свою выполнил', 
#     r'справился на отлично', r'со своими функциями справляется', r'задачу выполнил'
# ]
# gratitude_phrases = [
#     r'спасибо', r'рекомендую', r'советую', r'продавец молодец', r'благодарен', r'благодарю', r'советую к покупке',
#     r'спасибо большое', r'всем советую', r'спасибо за товар', r'спасибо продавцу', r'благодарю за товар', 
#     r'большое спасибо', r'очень благодарен', r'спасибо за доставку', r'огромное спасибо', r'спасибо за качественный товар', 
#     r'продавцу огромное спасибо', r'спасибо за оперативность', r'спасибо вам', r'благодарен за товар', 
#     r'спасибо, всё хорошо', r'продавец молодец', r'спасибо за хорошее обслуживание'
# ]
# delivery_phrases = [
#     r'пришел быстро', r'быстрая доставка', r'пришел вовремя', r'заказ пришел целый и вовремя', 
#     r'пришел целый', r'доставка вовремя', r'все пришло целым', r'товар пришел целым', r'пришел в срок',
#     r'пришел вовремя и целым', r'получил заказ вовремя', r'доставка - во!', r'все пришло как надо', 
#     r'пришел в полном порядке', r'отличная упаковка', r'все дошло целым', r'упаковано на совесть', 
#     r'крутая упаковка'
# ]
# confirmation_phrases = [
#     r'всё соответствует', r'всё как в описании', r'всё как заявлено', r'соответствует описанию', 
#     r'всё целое', r'всё в комплекте', r'всё норм', r'всё хорошо', r'как всегда', r'без проблем', 
#     r'нормально упаковано', r'нормально', r'всё норм'
# ]
# simple_statements_phrases = [
#     r'хорошая вещь', r'классная вещь', r'отличная вещь', r'удобно', r'нормально', r'работает', 
#     r'работает отлично', r'работает хорошо', r'всё нормально', r'всё работает', r'всё ок', 
#     r'всё окей', r'супер', r'класс', r'норм', r'отлично', r'хорошо', r'идеально', r'👍', r'👏', 
#     r'😆', r'🔥', r'💯', r'класс👍', r'все супер👍', r'👍👍👍', r'👍😊'
# ]


# # Определение масок и их эмбеддингов
# mask_names = [
#     "quality_phrases",
#     "functionality_phrases",
#     "gratitude_phrases",
#     "delivery_phrases",
#     "confirmation_phrases",
#     "simple_statements_phrases"
# ]

# mask_embeddings = [
#     compute_sentence_embeddings(gratitude_phrases),
#     compute_sentence_embeddings(delivery_phrases),
#     compute_sentence_embeddings(confirmation_phrases),
#     compute_sentence_embeddings(simple_statements_phrases),
#     compute_sentence_embeddings(quality_phrases),
#     compute_sentence_embeddings(functionality_phrases),
# ]

# # Вызов основной функции с эмбеддингами масок и логированием
# final_result = process_reviews(df_exploded, mask_embeddings, mask_names)

# # Показать результат
# display(final_result[['product', 'cluster_id', 'cluster_sentences', 'key_thought', 'word_count']])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Кластеризация для AG TECH / Антискотч спрей удалитель наклеек и скотча 210мл: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.51it/s]
Кластеризация для Autobrand_AED / Дополнительная led фара 30w с СТГ, ближний, дальний, 1 шт: 0it [00:00, ?it/s]                                                                            | 1/30 [00:07<03:27,  7.16s/it]
Кластеризация для CHAMELEON / Автошампунь для бесконтактной мойки автомобиля 5 л: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.00it/s]
Кластеризация для Detail / Iron Очиститель дисков, кузова от металлических вкраплен

KeyboardInterrupt: 

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.cluster import DBSCAN
# from sklearn.metrics.pairwise import cosine_similarity
# import torch
# from tqdm import tqdm

# # Устройство (GPU или CPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Функция для вычисления центра кластера (центроида)
# def find_centroid(embeddings):
#     return np.mean(embeddings, axis=0)

# # Функция для нахождения ключевой мысли в кластере
# def extract_key_thought(cluster_sentences):
#     sentences = cluster_sentences.split(" | ")
#     embeddings = compute_sentence_embeddings(sentences)
    
#     centroid = find_centroid(embeddings)
#     similarities = cosine_similarity(embeddings, [centroid])
#     key_sentence_index = np.argmax(similarities)
    
#     return sentences[key_sentence_index]

# # Функция для подсчета количества слов в каждом кластере
# def count_words(cluster_sentences):
#     words = cluster_sentences.split()
#     return len(words)

# # Функция для вычисления эмбеддингов
# def compute_sentence_embeddings(sentences):
#     inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
#     return embeddings

# # Функция для повторной кластеризации крупных кластеров
# def recluster_large_cluster(cluster_sentences, eps=0.1, min_samples=2):
#     sentences = cluster_sentences.split(" | ")
    
#     embeddings = compute_sentence_embeddings(sentences)
    
#     re_clustering = DBSCAN(eps=eps, min_samples=min_samples, metric="cosine").fit(embeddings)
    
#     re_cluster_dict = {}
#     for idx, label in enumerate(re_clustering.labels_):
#         if label == -1:
#             continue
#         label_str = str(label)
#         if label_str not in re_cluster_dict:
#             re_cluster_dict[label_str] = []
#         re_cluster_dict[label_str].append(sentences[idx])
    
#     return [" | ".join(cluster) for cluster in re_cluster_dict.values()]

# # Рекурсивная функция для кластеризации крупных кластеров
# def recursive_clustering(cluster_sentences, threshold, eps=0.25, min_samples=3, min_eps=0.05):
#     current_eps = eps
#     new_clusters = [cluster_sentences]

#     while True:
#         next_clusters = []
#         reclustered_any = False
        
#         for cluster in new_clusters:
#             if count_words(cluster) > threshold:
#                 while current_eps >= min_eps:
#                     reclustered = recluster_large_cluster(cluster, eps=current_eps, min_samples=min_samples)
#                     if len(reclustered) > 1:
#                         next_clusters.extend(reclustered)
#                         reclustered_any = True
#                         break  # Кластер успешно разделен, выходим из внутреннего цикла
#                     else:
#                         current_eps *= 0.9  # Уменьшаем eps и пробуем снова
                
#                 if len(reclustered) == 1:
#                     # Если кластер так и не был разделен, добавляем его обратно
#                     next_clusters.append(cluster)
#             else:
#                 next_clusters.append(cluster)
        
#         new_clusters = next_clusters
        
#         if not reclustered_any:
#             break
    
#     return new_clusters

# # Основной процесс кластеризации по товарам
# final_result = pd.DataFrame()

# for product_name, group in df_exploded.groupby('product'):
#     all_sentences = group['sentences'].tolist()

#     # Обработка предложений без разделения на батчи
#     all_embeddings = compute_sentence_embeddings(all_sentences)

#     # Прогресс-бар для начальной кластеризации
#     clustering = DBSCAN(eps=0.25, min_samples=3, metric="cosine").fit(all_embeddings)

#     cluster_dict = {}
#     for idx, label in tqdm(enumerate(clustering.labels_), desc=f"Organizing clusters for {product_name}"):
#         if label == -1:
#             continue
#         label_str = str(label)
#         if label_str not in cluster_dict:
#             cluster_dict[label_str] = set()
#         cluster_dict[label_str].add(all_sentences[idx])

#     clusters = [" | ".join(sentences) for sentences in cluster_dict.values()]
#     threshold = np.mean([count_words(cluster) for cluster in clusters]) * 1.5

#     final_clusters = []
#     for cluster in tqdm(clusters, desc="Recursive clustering"):
#         final_clusters.extend(recursive_clustering(cluster, threshold))

#     df_exploded_sorted = pd.DataFrame({'product': product_name, 'cluster_sentences': final_clusters})
#     df_exploded_sorted['word_count'] = df_exploded_sorted['cluster_sentences'].apply(count_words)
#     df_exploded_sorted['key_thought'] = df_exploded_sorted['cluster_sentences'].apply(extract_key_thought)

#     df_exploded_sorted = df_exploded_sorted.sort_values(by='word_count', ascending=False)

#     final_result = pd.concat([final_result, df_exploded_sorted], ignore_index=True)

# # Показать результат
# display(final_result[['product', 'cluster_sentences', 'key_thought', 'word_count']])


Organizing clusters for *Happy Family* / Очки для вождения. Авиаторы 2 шт: 129it [00:00, 833690.63it/s]
Recursive clustering: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.41s/it]
Organizing clusters for AutoVirazh / Компрессор автомобильный двухпоршневой 85л мин: 4it [00:00, 55007.27it/s]
/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
Recursive clustering: 0it [00:00, ?it/s]
Organizing clusters for Clements / Вкладыш для автодокументов прозрачный 100мкм обложка: 220it [00:00, 851242.51it/s]
Recursive clustering: 100%|████████████████████████████████████████████████████████

,product,cluster_sentences,key_thought,word_count
0,*Happy Family* / Очки для вождения. Авиаторы 2 шт,"Свои функции выполняют, особенно жёлтые линзы ...","Хлипкие после протирке одно стекло выволелось,...",53
1,*Happy Family* / Очки для вождения. Авиаторы 2 шт,"Всё замечательно, спасибо | Нормально | Отличн...",Всё отлично,10
2,*Happy Family* / Очки для вождения. Авиаторы 2 шт,"Детские | Маленькие очень, детские | Ну очень ...","Маленькие очень, детские",9
3,Clements / Вкладыш для автодокументов прозрачн...,"Очень удобный вкладыш, все документы поместили...","Отличный вкладыш, плотный, по размеру подошёл",152
4,Clements / Вкладыш для автодокументов прозрачн...,Рекомендую! | Качество хорошее! | Качество хор...,Качество отличное,77
...,...,...,...,...
163,Сезон товаров / Умные антибликовые очки ночног...,Мне понравились Рекомендую | Мужу понравились ...,Мне понравились Рекомендую,10
164,Сезон товаров / Умные антибликовые очки ночног...,Огромное спасибо!!! | Спасибо!) | Спасибо! | С...,Спасибо!👍,8
165,Сезон товаров / Умные антибликовые очки ночног...,Супер . | Супер! | Отлично! | Отлично,Отлично!,8
166,Сезон товаров / Умные антибликовые очки ночног...,ОТЛИЧНЫЕ | шикарные | Очень хороший,ОТЛИЧНЫЕ,6


In [ ]:
# import re
# import emoji
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np
# import torch
# from transformers import AutoTokenizer, AutoModel
# from nltk.corpus import stopwords

# # Установка стоп-слов
# nltk.download('stopwords')
# stop_words = set(stopwords.words('russian'))

# # Загрузка модели spaCy для русского языка
# nlp = spacy.load("ru_core_news_lg")

# # Функция для проверки наличия эмодзи в строке
# def contains_emoji(text):
#     return any(char in emoji.EMOJI_DATA for char in text)

# # Существующие маски
# common_phrases = [
#     r'всё ок', r'супер', r'класс', r'нормально', r'норм', r'всё норм', r'отлично', r'хорошо', r'нормально упаковано',
#     r'без проблем', r'как всегда', r'норм'
# ]
# emotional_phrases = [
#     r'спасибо', r'рекомендую', r'советую', r'продавец молодец', r'молодец', r'рекомендую продавца', r'благодарен', r'благодарю',
#     r'советую к покупке', r'спасибо большое', r'всем советую'
# ]
# short_phrases = [
#     r'пришел быстро', r'уже брал', r'помогло', r'не помогло', r'пока не пробовал', r'отличная вещь', r'всё окей',
#     r'нормально', r'быстрая доставка', r'пришел вовремя'
# ]
# item_phrases = [
#     r'хорошая вещь', r'классная вещь', r'отличная вещь', r'нужная вещь', r'удобная вещь', r'полезная вещь',
#     r'прекрасная вещь', r'замечательная вещь', r'хороший продукт', r'отличный продукт', r'качественная вещь'
# ]
# task_phrases = [
#     r'с задачей справился', r'с функциями справился', r'задачу свою выполнил', r'справился на отлично', 
#     r'функции выполняет', r'с задачей справляется', r'задачу выполнил', r'справляется с задачей', 
#     r'со своими функциями справляется', r'справился с задачей'
# ]
# delivery_phrases = [
#     r'заказ пришел целый и вовремя', r'пришел вовремя', r'пришел целый', r'доставка вовремя', r'все пришло целым', 
#     r'товар пришел целым', r'пришел в срок', r'доставка быстрая', r'пришел вовремя и целым', r'получил заказ вовремя'
# ]
# emoji_phrases = [
#     r'идеально', r'отлично', r'👍', r'👏', r'😆', r'🔥', r'💯', r'класс', r'класс👍', r'все супер👍', r'👍👍👍', r'👍😊'
# ]
# negative_condition_phrases = [
#     r'пришло все побитое', r'упаковка порвана', r'всё сломано', r'товар треснул', r'получил товар с дефектом', 
#     r'погнутая упаковка', r'пришло разорванное', r'все разлито', r'коробка помята', r'всё побилось', 
#     r'сломанный товар', r'все порвано', r'пришел весь в трещинах', r'поврежденная упаковка', r'товар не работает'
# ]
# positive_condition_phrases = [
#     r'всё пришло целое и невредимое', r'доставка - во!', r'крутая упаковка', r'упаковано на совесть', 
#     r'все пришло в идеальном состоянии', r'товар в отличном состоянии', r'без повреждений', r'упаковка целая', 
#     r'товар без дефектов', r'все пришло как надо', r'пришел в полном порядке', r'отличная упаковка', 
#     r'все дошло целым', r'доставка без повреждений', r'идеальное состояние'
# ]
# gratitude_phrases = [
#     r'спасибо за товар', r'спасибо продавцу', r'спасибо большое', r'благодарю за товар', r'большое спасибо', 
#     r'очень благодарен', r'спасибо за доставку', r'огромное спасибо', r'спасибо за качественный товар', 
#     r'продавцу огромное спасибо', r'спасибо за оперативность', r'спасибо вам', r'благодарен за товар', 
#     r'спасибо, всё хорошо', r'продавец молодец', r'спасибо за хорошее обслуживание'
# ]
# neutral_quality_phrases = [
#     r'всё отлично', r'всё хорошо', r'все супер', r'очень доволен покупкой', r'работает хорошо', 
#     r'надеюсь прослужить долго', r'всё целое', r'всё в комплекте', r'всё как в описании', 
#     r'всё как заявлено', r'за свою цену отлично', r'качество хорошее', r'отличное качество', 
#     r'комплект как в описании', r'мелочь, а приятно', r'мне всё понравилось', r'добрый день', 
#     r'всё соответствует', r'работает хорошо, спасибо', r'всё супер 👌'
# ]

# # Новые маски
# confirmation_phrases = [
#     r'всё соответствует', r'всё как в описании', r'всё как заявлено', r'соответствует описанию', r'всё целое', r'всё в комплекте', r'всё норм', r'всё хорошо'
# ]
# simple_statements_phrases = [
#     r'хорошая вещь', r'классная вещь', r'отличная вещь', r'удобно', r'нормально', r'работает', r'работает отлично', r'работает хорошо', r'всё нормально', r'всё работает'
# ]
# quality_phrases = [
#     r'качество хорошее', r'отличное качество', r'качественно', r'прекрасное качество', r'высокое качество', r'качественный товар', r'качество отличное', r'качество удовлетворительное'
# ]
# functionality_phrases = [
#     r'работает отлично', r'работает хорошо', r'всё работает', r'функции выполняет', r'функциональный', r'функции справляются', r'с задачей справился', r'справляется с задачей', r'функции выполняет'
# ]
# price_phrases = [
#     r'цена нормальная', r'цена адекватная', r'соотношение цена/качество', r'цена отличная', r'цена хорошая', r'цена приемлемая', r'цена оправдана', r'цена низкая', r'цена высокая', r'соотношение цены и качества', r'за такую цену', r'вполне приемлемая цена'
# ]
# durability_phrases = [
#     r'надеюсь прослужить долго', r'пользуюсь долго', r'надежный товар', r'долговечный', r'хватит надолго', r'буду использовать долго', r'на сезон хватит', r'долго пользуюсь', r'проверено временем', r'выдерживает нагрузки', r'посмотрим, сколько продержится'
# ]
# appearance_phrases = [
#     r'выглядит хорошо', r'смотрится красиво', r'внешний вид отличный', r'стильно выглядит', r'выглядит красиво', r'смотрится отлично', r'внешне приятно', r'стильный', r'выглядит качественно'
# ]

# # Функция для вычисления эмбеддингов
# def compute_sentence_embeddings(sentences):
#     inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

# # Определение масок и их эмбеддингов
# gratitude_emb = compute_sentence_embeddings(gratitude_phrases)
# common_emb = compute_sentence_embeddings(common_phrases)
# emotional_emb = compute_sentence_embeddings(emotional_phrases)
# short_emb = compute_sentence_embeddings(short_phrases)
# item_emb = compute_sentence_embeddings(item_phrases)
# task_emb = compute_sentence_embeddings(task_phrases)
# delivery_emb = compute_sentence_embeddings(delivery_phrases)
# emoji_text_emb = compute_sentence_embeddings(emoji_phrases)
# negative_condition_emb = compute_sentence_embeddings(negative_condition_phrases)
# positive_condition_emb = compute_sentence_embeddings(positive_condition_phrases)
# neutral_quality_emb = compute_sentence_embeddings(neutral_quality_phrases)
# confirmation_emb = compute_sentence_embeddings(confirmation_phrases)
# simple_statements_emb = compute_sentence_embeddings(simple_statements_phrases)
# quality_emb = compute_sentence_embeddings(quality_phrases)
# functionality_emb = compute_sentence_embeddings(functionality_phrases)
# price_emb = compute_sentence_embeddings(price_phrases)
# durability_emb = compute_sentence_embeddings(durability_phrases)
# appearance_emb = compute_sentence_embeddings(appearance_phrases)

# # Функция для проверки семантической близости с каждой маской
# def is_similar_to_mask(key_thought, mask_emb):
#     key_emb = compute_sentence_embeddings([key_thought])
#     return np.max(cosine_similarity(key_emb, mask_emb)) > 0.65  # Порог близости можно настроить

# # Проверка ключевых мыслей на семантическую близость к каждой маске
# final_result['is_similar_to_emoji'] = final_result['key_thought'].apply(lambda x: contains_emoji(x) or is_similar_to_mask(x, emoji_text_emb))
# final_result['is_similar_to_common'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, common_emb))
# final_result['is_similar_to_emotional'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, emotional_emb))
# final_result['is_similar_to_short'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, short_emb))
# final_result['is_similar_to_item'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, item_emb))
# final_result['is_similar_to_task'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, task_emb))
# final_result['is_similar_to_delivery'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, delivery_emb))
# final_result['is_similar_to_negative_condition'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, negative_condition_emb))
# final_result['is_similar_to_positive_condition'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, positive_condition_emb))
# final_result['is_similar_to_gratitude'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, gratitude_emb))
# final_result['is_similar_to_neutral_quality'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, neutral_quality_emb))
# final_result['is_similar_to_confirmation'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, confirmation_emb))
# final_result['is_similar_to_simple_statements'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, simple_statements_emb))
# final_result['is_similar_to_quality'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, quality_emb))
# final_result['is_similar_to_functionality'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, functionality_emb))
# final_result['is_similar_to_price'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, price_emb))
# final_result['is_similar_to_durability'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, durability_emb))
# final_result['is_similar_to_appearance'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, appearance_emb))

# # Удаление пустых кластеров
# final_result = final_result[final_result['cluster_sentences'].str.strip().astype(bool)]

# # Слова для удаления кластеров
# exclusion_words = [
#     r'отличный', r'хороший', r'шикарный', r'офигенный', r'замечательный', r'потрясающий', r'великолепный', 
#     r'прекрасный', r'изумительный', r'фантастический', r'удивительный', r'невероятный', r'зачётный', r'суперский', 
#     r'классный', r'крутой', r'понравилось', r'понравились', r'люблю', r'восхищён', 
#     r'доволен', r'наслаждаюсь', r'порадовало'
# ]

# # Функция для лемматизации текста
# def lemmatize_text(text):
#     doc = nlp(text)
#     return " ".join([token.lemma_ for token in doc])


# # Предварительное вычисление эмбеддингов для лемматизированных слов из списка exclusion_words
# lemmatized_exclusion_words = [lemmatize_text(word) for word in exclusion_words]
# exclusion_emb = compute_sentence_embeddings(lemmatized_exclusion_words)

# # Обновленная функция для проверки с использованием семантической близости
# def is_single_word_or_stop_word(key_thought):
#     words = re.findall(r'\w+', key_thought)  # Извлекаем все слова
#     if len(words) == 1:
#         return True
#     if len(words) == 2 and words[1] in stop_words:
#         return True
#     if len(words) == 2 and re.match(r'[^\w\s]', words[1]):  # Пунктуация как второе слово
#         return True
#     if len(words) in [2, 3]:
#         lemmatized_key_thought = lemmatize_text(key_thought)
#         lemmatized_words = re.findall(r'\w+', lemmatized_key_thought)
#         for word in lemmatized_words:
#             key_emb = compute_sentence_embeddings([word])
#             max_similarity = np.max(cosine_similarity(key_emb, exclusion_emb))
#             if max_similarity > 0.9:  # Порог близости можно настроить
#                 print(f"Близость - {max_similarity}. Исключаем {key_thought}")
#                 return True
#     return False

# # Применение фильтрации
# final_result = final_result[~final_result['key_thought'].apply(is_single_word_or_stop_word)]

# # Обновление фильтрации кластеров, где все маски False
# mask_false_clusters = (
#     ~final_result['is_similar_to_emoji'] &
#     ~final_result['is_similar_to_common'] &
#     ~final_result['is_similar_to_emotional'] &
#     ~final_result['is_similar_to_short'] &
#     ~final_result['is_similar_to_item'] &
#     ~final_result['is_similar_to_task'] &
#     ~final_result['is_similar_to_delivery'] &
#     ~final_result['is_similar_to_negative_condition'] &
#     ~final_result['is_similar_to_positive_condition'] &
#     ~final_result['is_similar_to_gratitude'] &
#     ~final_result['is_similar_to_neutral_quality'] &
#     ~final_result['is_similar_to_confirmation'] &
#     ~final_result['is_similar_to_simple_statements'] &
#     ~final_result['is_similar_to_quality'] &
#     ~final_result['is_similar_to_functionality'] &
#     ~final_result['is_similar_to_price'] &
#     ~final_result['is_similar_to_durability'] &
#     ~final_result['is_similar_to_appearance']
# )

# # Вывод результатов
# df_false_clusters = final_result[mask_false_clusters]
# display(df_false_clusters[['product', 'cluster_sentences', 'key_thought', 'word_count']])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Близость - 0.9945003986358643. Исключаем Всё отлично
Близость - 0.9945003986358643. Исключаем Качество отличное
Близость - 0.9945003986358643. Исключаем Отличный, прочный
Близость - 0.9945003986358643. Исключаем Все отлично
Близость - 0.9945003986358643. Исключаем Отличный компрессор!
Близость - 0.9945003986358643. Исключаем Отличные очки
Близость - 0.9945003986358643. Исключаем Все отлично
Близость - 0.9945003986358643. Исключаем Отличные очки
Близость - 0.9945003986358643. Исключаем Всё отлично
Близость - 0.9945003986358643. Исключаем Отличный товар
Близость - 0.9553651809692383. Исключаем Мне понравились
Близость - 0.9945003986358643. Исключаем Отличные очки
Близость - 0.9553651809692383. Исключаем Мужу понравились спасибо
Близость - 0.9777071475982666. Исключаем Хорошие очки Спасибо
Близость - 0.9945003986358643. Исключаем Всё отлично
Близость - 0.9922985434532166. Исключаем Классный аппарат, рекомендую
Близость - 0.9553651809692383. Исключаем Мужу очень понравились
Близость - 0.99

,product,cluster_sentences,key_thought,word_count
0,*Happy Family* / Очки для вождения. Авиаторы 2 шт,"Свои функции выполняют, особенно жёлтые линзы ...","Хлипкие после протирке одно стекло выволелось,...",53
2,*Happy Family* / Очки для вождения. Авиаторы 2 шт,"Детские | Маленькие очень, детские | Ну очень ...","Маленькие очень, детские",9
6,Clements / Вкладыш для автодокументов прозрачн...,"Вместились все документы: ТС, страховые, паспо...","Отличный вкладыш, плотный, все документы вошли...",43
14,Eternal way / Беспроводной автомобильный аккум...,"Заказала себе компрессор, меня подкупил необыч...",Хороший компрессор Брала в подарок мужу Сразу ...,43
17,FST Auto / Очки для вождения. Классика 2 шт,"Как игрушечные, лёгкие, но качество соответств...","Сколько стоят, так и выглядят: дёшево и просто",37
28,FST Auto / Очки для вождения. Спортивный стиль...,"Как игрушечные, лёгкие, но качество соответств...","Сколько стоят, так и выглядят: дёшево и просто",37
43,Grand House / Очки для водителя антиблик,"Годные очки, ночью как днём едешь, днём солнце...","Пол дня сегодня ехал в желтых очках, глаза не ...",65
44,Grand House / Очки для водителя антиблик,"Очки хорошие, мужу понравился, рекомендую | Оч...","Очки хорошие, понравились, рекомендую",33
52,Lieblich Hause (Haz) / Компрессор воздушный бе...,"Есть встроенный манометр, соответственно не на...",Накачал 2 колеса для велика до 3 атмосфер - сп...,173
53,Lieblich Hause (Haz) / Компрессор воздушный бе...,Купил качать ватрушку и для велосипедов Работа...,"Покупал для велосипеда, отличная штука Пробова...",85


In [ ]:
# df_false_clusters[['cluster_sentences', 'key_thought', 'word_count']].to_csv("./reviews_keywords/clusters.csv")

In [ ]:
# import torch
# from transformers import T5ForConditionalGeneration, T5Tokenizer
# from tqdm import tqdm

# # Определение устройства (GPU или CPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Загрузка модели T5
# model_name = "cointegrated/rut5-base-multitask"  # Модель для русской T5
# model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
# tokenizer = T5Tokenizer.from_pretrained(model_name)

# # Функция для разбиения текста на части
# def chunk_text(text, max_length=100):
#     words = text.split()
#     chunks = [' '.join(words[i:i + max_length]) for i in range(0, len(words), max_length)]
#     return chunks

# # Функция для суммаризации текста с настройкой параметров генерации
# def summarize_text(text):
#     # Токенизация и перенос на GPU
#     inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True).to(device)
    
#     # Генерация суммаризации с использованием GPU
#     summary_ids = model.generate(
#         inputs.input_ids, 
#         max_length=150, 
#         min_length=40, 
#         length_penalty=4,  # Увеличиваем penalty для избежания повторений
#         num_beams=16,  # Увеличиваем количество beam для улучшения качества
#         repetition_penalty=3.0,  # Добавляем штраф за повторения
#         early_stopping=True
#     )
    
#     # Перенос результата обратно на CPU и декодирование
#     return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# # Функция для суммаризации длинных текстов с рекурсивным подходом
# def recursive_summarization(text, depth=2):
#     chunks = chunk_text(text, max_length=100)  # Разбиение текста на части, каждая до 100 слов
#     summaries = [summarize_text(chunk) for chunk in chunks]
    
#     # Если достигли необходимой глубины рекурсии, возвращаем результат
#     if depth <= 1:
#         return ' '.join(summaries)
    
#     # В противном случае суммаризируем еще раз на более высокой глубине
#     return recursive_summarization(' '.join(summaries), depth - 1)

# # Применение рекурсивной суммаризации к каждому кластеру с прогресс-баром и использованием GPU
# df_false_clusters['summary'] = [
#     recursive_summarization(text, depth=2) for text in tqdm(df_false_clusters['cluster_sentences'], desc="Summarizing clusters")
# ]

# # Вывод результатов суммаризации
# display(df_false_clusters[['cluster_sentences', 'summary']])


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Summarizing clusters: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [02:09<00:00,  2.09s/it]


,cluster_sentences,summary
0,А вот в темных ездить по слепящей от солнца до...,В темных очках чуть комфортнее ездить по слепя...
1,Одно стекло светлое другое черное вообще разны...,"Одно стекло светлое, другое черное...<br>Стекл..."
7,"Маленькие очень, детские | Детские | Ну очень ...","Маленькие очень, детские <br><br>Малыши очень ..."
10,Качественная обложка! | И по размеру идеально ...,"Немного не влез в обложку, пришлось подрезать ..."
17,"Внутренняя поверхность текстурная, поэтому лам...","Внутренняя поверхность текстурная, поэтому лам..."
...,...,...
244,Покупала папе- рыбаку. | Покупала мужу. | Брал...,Покупала папе- рыбаку <br>Покупала мужу <br>Бы...
249,Очки хорошие дедушке понравились | Фото соотве...,"Очки хорошие, широкие, мне понравились очки. <..."
250,"Очки хорошие,сваи функции выполняют на 5+ .Мин...","Хорошие очки не мешают обзору, они какие то ра..."
256,"ОГРОМНАЯ БЛАГОДАРНОСТЬ ЗА ОЧКИ , МУЖУ ОЧЕНЬ П...","Спасибо, брала мужу, сказал отличные очки. В д..."


In [ ]:
# import re
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# # Загрузка модели и токенайзера для коррекции текста
# model_name = "cointegrated/rut5-base-multitask"  # Модель T5 для мультитаскинга на русском языке
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# # Функция для удаления повторов и объединения текста
# def clean_text(text):
#     sentences = text.split('<br>')
#     cleaned_sentences = []
#     seen = set()
    
#     for sentence in sentences:
#         sentence = sentence.strip()
#         if sentence not in seen:
#             cleaned_sentences.append(sentence)
#             seen.add(sentence)
    
#     return ' '.join(cleaned_sentences)

# # Функция для корректировки текста с использованием T5
# def correct_text(text):
#     inputs = tokenizer("correct: " + text, return_tensors="pt", max_length=512, truncation=True).to(device)
#     summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=40, length_penalty=1.0, num_beams=4, early_stopping=True)
#     return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# # Пример текста
# text = 'Хороший очиститель, чистил дроссель старой Тойоты, отмывает очень хорошо, спасибо продавцу. <br>Очень хорошо отмывает загрязнения, нагар Хороший очиститель, прешёл хорошо упакован, спасибо продавцу <br>Хороший герметик, помог. <br>Хороший герметик, помог. <br>Отличный товар рекомендую'

# # Очистка текста от повторов
# cleaned_text = clean_text(text)

# # Корректировка текста для улучшения согласованности и пунктуации
# final_text = correct_text(cleaned_text)
# final_text


/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


'Хороший герметик, помог. Хороший герметик, помог. Хороший герметик, помог. Хороший герметик, помог. Хороший герметик, помог.'

In [ ]:
# df_exploded_sorted["cluster_sentences"].to_csv("./reviews_keywords/clusters.csv")

In [ ]:
# def filter_tokens(syntax_analysis):
#     # Отключаем некоторые фильтры для проверки
#     filtered_tokens = [
#         token for token in syntax_analysis 
#         if token[1] not in {"PUNCT", "SPACE"}  # Исключаем только знаки препинания и пробелы
#         # Отключаем фильтрацию по длине
#     ]
    
#     return filtered_tokens

# def extract_key_phrases_from_sentences(doc):
#     key_phrases = []
    
#     for sent in doc.sents:
#         syntax_analysis = [(token.text, token.pos_, token.dep_, token.head.text) for token in sent]
#         filtered_tokens = filter_tokens(syntax_analysis)
#         phrase = []

#         for i, token in enumerate(filtered_tokens):
#             if token[1] in {"NOUN", "VERB"}:  # Существительное или глагол
#                 if phrase:
#                     key_phrases.append(" ".join(phrase))
#                     phrase = []
#                 phrase.append(token[0])
#             elif token[1] in {"ADJ", "ADV"}:  # Прилагательные, наречия
#                 if phrase:
#                     phrase.append(token[0])

#             # Если конец текста или следующая часть речи не связана с текущей фразой
#             if i == len(filtered_tokens) - 1 or filtered_tokens[i+1][1] not in {"ADJ", "ADV", "ADP", "CCONJ", "SCONJ", "PART"}:
#                 if phrase:
#                     key_phrases.append(" ".join(phrase))
#                     phrase = []
#     key_phrases = [phrase for phrase in key_phrases if len(phrase.split()) > 1 and len(phrase.strip()) > 2]

#     return " ".join(key_phrases)


# def extract_key_phrases_from_clusters(clusters):
#     key_phrases = []
#     for cluster in clusters:
#         cluster_key_phrases = []
#         for sentences in cluster:  # Так как cluster теперь список списков
#             doc = nlp(sentences)
#             cluster_key_phrases.append(extract_key_phrases_from_sentences(doc))
#         key_phrases.append(" ".join(cluster_key_phrases))  # Соединяем все ключевые фразы из одного кластера в одну строку
#     return key_phrases

# # Применение функции
# dataset = dataset.map(lambda batch: {"key_phrases": extract_key_phrases_from_clusters(batch['clusters'])}, batched=True, batch_size=8)


# # Частотный анализ по ключевым фразам
# key_phrases = dataset['key_phrases']
# phrase_freq = Counter(key_phrases)

# # Вывод результатов
# print("Частотный анализ ключевых фраз (по семантической близости):")
# print(phrase_freq.most_common(10))

Map:   0%|          | 0/5574 [00:00<?, ? examples/s]

KeyError: 'clusters'

In [ ]:
# df.to_csv("./reviews_keywords/temp_spacy.csv", index=False)